Analyze GPU project operator in Spark event logs.

In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd

cpu_dir = "/home/rou/src/spark-rapids-tools/dataproc-cpu-3k"
gpu_dir = "/home/rou/src/spark-rapids-tools/dataproc-gpu-3k"
cpu_files = os.listdir(cpu_dir)
cpu_files.sort()
gpu_files = os.listdir(gpu_dir)
gpu_files.sort()

In [ ]:
project_times = []
for i, app in enumerate(gpu_files):
    sql_info = pd.read_csv(gpu_dir + "/" + app + "/sql_plan_metrics_for_application.csv")
    filtered_df = sql_info[(sql_info["name"] == "op time") & (sql_info["nodeName"] == "GpuProject")].copy()
    filtered_df["appIndex"] = i 
    filtered_df["app"] = app 
    project_times.append(filtered_df) 
project_times_df = pd.concat(project_times)
project_times_df["total milliseconds"] = project_times_df["total"] / 1000000
project_times_df[project_times_df["appIndex"] == 2]

In [ ]:
codegen_times = []
for i, app in enumerate(cpu_files) :
    sql_info = pd.read_csv(cpu_dir + "/" + app + "/sql_plan_metrics_for_application.csv")
    filtered_df = sql_info[(sql_info["name"] == "duration") & sql_info["nodeName"].str.startswith("WholeStageCodegen")].copy()
    filtered_df["appIndex"] = i  
    filtered_df["app"] = app 
    codegen_times.append(filtered_df)
codegen_times_df = pd.concat(codegen_times)
codegen_times_df

In [ ]:
mappings = []
for i, app in enumerate(cpu_files):
    mapping_info = pd.read_csv(cpu_dir + "/" + app + "/wholestagecodegen_mapping.csv")
    filtered_df = mapping_info[mapping_info["Child Node"] == "Project"].copy()
    filtered_df["appIndex"] = i  
    filtered_df["app"] = app 
    mappings.append(filtered_df)
mapping_df = pd.concat(mappings)
mapping_df[mapping_df["appIndex"] == 5]

In [ ]:
combined_codegen = pd.merge(codegen_times_df, mapping_df, on=["appIndex", "sqlID", "nodeID"], how="right")
combined_codegen

In [ ]:
project_counts = project_times_df.groupby("appIndex").size().reset_index(name="counts gpu")
codegen_counts = combined_codegen.groupby("appIndex").size().reset_index(name="counts cpu")
combined_counts = pd.merge(project_counts, codegen_counts, on="appIndex", how="outer")
combined_counts[combined_counts["counts gpu"] != combined_counts["counts cpu"]]

2 - query 24a: GPU misisng 1 `ReusedExchange`
3 - query 24b: GPU misisng 1 `ReusedExchange`
5 - query 47:
  CPU: 14 `Project`s - 6 `ReusedExchange`s (3+3+0+0+0+0) = 8
  GPU: 15 (extra project after window) `GpuProject`s - 3 `ReusedExchange`s (3+3+0) = 9
6 - query 56:
  CPU: 21 `Project`s - 5 out of 7 `ReusedExchange`s - 2 `ReusedSubquery` = 14
  GPU: 21 `GpuProject`s - 5 out of 7 `ReusedExchange`s - 2 `ReusedSubquery` = 14???
